In [ ]:
import os
import pandas as pd
from typing import List

In [ ]:
from {{package_name}} import utils
from {{package_name}}.models_training import utils_models

blabla général sur les frameworks

# Entrainement d'un modèle de détection d'auteur 'out of the box'

On va partir de textes de différents auteurs du XIXème siècle et fabriquer un modèle permettant d'identifier automatiquement l'auteur

In [ ]:
# On charge les textes
data_path = utils.get_data_path()
df_texts = pd.read_csv(os.path.join(data_path, 'texts.csv'), sep=';')

Nous allons tout d'abord couper notre dataset en deux parties, un ensemble d'entrainement et un ensemble de test afin de pouvoir, entraîner et tester notre modèle. Notez que nous ne fabriquons pas d'ensemble de validation car nous n'allons pas tuner notre modèle. 

Pour cela, nous allons utiliser les scripts contenus dans le package. Pour couper en deux notre dataset, procédez de la manière suivante :

Lancez un terminal
Activez votre environnement virtuel
Placer vous dans votre package dans le répertoire suivant:

`cd {{package_name}}-scripts/utils`

Appelez le script de découpe du dataset pour réaliser un 'stratified split': 

`python 0_split_train_valid_test.py -f texts.csv --perc_train 0.6 --perc_valid 0.0 --perc_test 0.4`

Si par hasard, vous avez fait une fausse manip et qu'il faut regénérez des fichiers de train/test, rajoutez l'argument `--overwrite` pour écrasez les fichiers erronés

Vous pouvez alors voir que deux fichiers, `texts_train.csv` et `texts_test.csv` ont été créés dans `{{package_name}}-data`

Vous pouvez regarder à quoi ils ressemblent en utilisant la cellule ci-dessous pour les charger dans le notebook:

In [ ]:
df_train, metadata_train = utils.read_csv(os.path.join(data_path, 'texts_train.csv'))
df_test, metadata_test = utils.read_csv(os.path.join(data_path, 'texts_test.csv'))

Nous allons maintenant faire une étape de préprocessing sur les textes afin de les normaliser et que le modèle performe mieux. Pour cela, nous allons utiliser le script `1_preprocess_data.py` situé ici : `{{package_name}}-scripts`

Pour cela :

Lancez un terminal
Activez votre environnement virtuel
Placer vous dans votre package dans le répertoire suivant:

`cd {{package_name}}-scripts`

Appelez le script de préprocessing: 

`python 1_preprocess_data.py -f texts_train.csv texts_test.csv --input_col text`

Vous pouvez remarquer que nous devons préciser la colonne sur laquelle doit agir le préprocessing. Nous pouvons voir le résultat du preprocessing sur l'ensemble de test. Notez qu'une nouvelle colonne a été ajouté, la colonne 'preprocessed_text' qui contient le texte modifié.

In [ ]:
df_test_preprocessed, metadata_test_preprocessed = utils.read_csv(os.path.join(data_path, 'texts_test_preprocess_P1.csv'))

Le préprocessing appliqué est celui fourni de base par le package mais il est très facile d'ajouter ses propres pipelined de preprocessing . Pour cela, ouvrez le fichier : `{{package_name}}/preprocessing/preprocess.py`. Dans ce module, vous voyez que l'ensemble des pipelines de preprocessing sont renseignés dans la fonction `get_preprocessors_dict`. C'est ici que vous pouvez rajouter vos pipelines personnalisées. 

On peut également jeter un coup d'oeil à la fonction `preprocess_sentence_P1` qui indique les différentes étapes de préprocessing (pour plus d'informations, il suffit de lire la documentation du package words_n_fun, une librairie facilitant tout le travail de manipulation de texte).

Si vous avez l'oeil, vous pouvez remarquer que les accents n'ont pas été traités correctement pas la pipeline `preprocess_sentence_P1`. On va donc corrigez cela. Pour ce faire, il suffit de rajouter `'remove_accents'` en première étape dans la pipeline de la fonction `preprocess_sentence_P1`. Puis relancez : 

`python 1_preprocess_data.py -f texts_train.csv texts_test.csv --input_col text`

Les accents sont maintenant correctement traités:

In [ ]:
df_test_preprocessed, metadata_test_preprocessed = utils.read_csv(os.path.join(data_path, 'texts_test_preprocess_P1.csv'))

Nous allons maintenant entraîner notre modèle en utilisant le script `2_training.py`

Pour cela :

Lancez un terminal
Activez votre environnement virtuel
Placer vous dans votre package dans le répertoire suivant:

`cd {{package_name}}-scripts`

Appelez le script d'entrainement: 

`python 2_training.py -f texts_train_preprocess_P1.csv --x_col preprocessed_text --y_col author`

Cela va appliquer un simple TF-IDF + SVM sur les données

Nous pourrions utiliser ensuite le script `3_predict.py` pour utiliser le modèle pour prédire sur notre ensemble de test mais nous allons procéder autrement pour avoir les résultats directement dans ce notebook. Si vous allez dans `{{package_name}}-models/model_tfidf_svm` vous allez voir un dossier du type `model_tfidf_svm_{YYYY_MM_DD-hh_mm_ss}` qui contient la sauvegarde du modèle que nous venons d'entraîner. 

Vous pouvez par exemple voir le f1_score sur l'ensemble d'entrainement, la matrice de confusion sur l'ensemble d'entraînement etc. Notez que nous aurions pu spécifier un dataset de validation au script `2_training.py` et nous aurions alors aussi accès à des métriques sur ce dataset. 

Ce dossier contient également les fichiers de sauvegarde du modèle que nous allons charger dans la cellule suivante (en remplacant bien sûr `{YYYY_MM_DD-hh_mm_ss}` par les nombres correspondant

In [ ]:
model, model_conf = utils_models.load_model('model_tfidf_svm_{YYYY_MM_DD-hh_mm_ss}')

Voilà, le modèle est chargé, prêt à être utilisé. Ce que nous allons directement faire sur l'ensemble de test chargé plus haut :

In [ ]:
df_test_preprocessed['prediction'] = model.predict(df_test_preprocessed['preprocessed_text'])
print('accuracy :', len(df_test_preprocessed[df_test_preprocessed['prediction']==df_test_preprocessed['author']])/len(df_test_preprocessed))

Mais, il est tout nul ce modèle !!!!

Pour remédier à ça, nous pourrions essayer d'autres modèles qui sont déjà packagés dans le projet. 

Mais nous allons utiliser une autre manière de faire qui illustre toute la puissance de ces frameworks. Nous allons créer notre propre modèle qui incorporera tous les à-côtés que nous avons vu (sauvegarde automatique des modèles, chargement des modèles, calcul des métriques...). 

La logique que nous allons appliquer est simple. Au lieu d'appliquer un TF-IDF + SVM à des livres entiers, nous allons découper les livres en 'phrases' et faire apprendre le modèle sur ces 'phrases'. Pour prédire l'auteur d'un livre, nous allons donc découper en 'phrases', prédire pour chaque 'phrase' un auteur et prendre pour prédiction finale, l'auteur qui a le plus de 'phrases' lui correspondant.

On doit donc créer d'abord une fonction qui prend un texte et qui le découpe en phrase. On va également lui indiquer combien de mots par phrase on souhaite. Compléter la fonction suivante :

In [ ]:
def text_to_sentences(text: str, nb_word_sentence: int) -> List[str]:
    '''Transforms a text in sentences.

    Args:
        text (str) : The text to cut in sentences
        nb_word_sentence (int) : The number of words in a sentence
    Returns:
        list : A list of sentences
    
    '''
    

Une fois la fonction ci-dessus complétée, vous pouvez ouvrir le fichier `utils_tutorial_fr.py` situé au même endroit que ce notebook. A l'intérieur, vous trouverez une solution possible pour la fonction ci-dessus (vous pouvez la remplacer par votre version si vous le souhaitez). 

Le reste du fichier contient notre modèle de prédiction d'auteurs: 

La fonction `df_texts_to_df_sentences` permet de formater une liste de textes et d'auteur en un dataframe de 'phrases' (en utilisant la fonction `text_to_sentences`). Il s'agit uniquement d'un fonction de formattage et n'apporte aucune réelle plus-value.

La classe `ModelAuthor` est la classe implémentant notre modèle. Notez qu'elle descend de la classe `ModelTfidfSvm`, ce qui va nous permettre très simplement d'avoir accès à toutes les méthodes voulues. 

Dans l'`__init__`, on rajoute simplement `nb_word_sentence`, l'argument de la fonction ci-dessus.

Dans le `fit`, on ajoute simplement le découpage en 'phrases' avant d'appeler le même modèle TF-IDF que précedemment.

Dans le `predict` on utilise la méthode `predict_author` qui implémente notre méthode de découpage en 'phrases' et en vote majoritaire pour un auteur.

Ainsi en moins d'une centaine de ligne de code, nous avons développé un nouveau modèle qui s'incorpore totalement dans le projet et qui va pouvoir utiliser tous ses outils

Nous allons donc utiliser notre nouveau modèle en faisant les mêmes étapes que précedemment (sauf les étapes de préprocessing du texte qui ne changent pas). 

On va tout d'abord incorporez notre modèle au projet. Pour ce faire, copier le fichier `utils_tutorial_fr.py` à l'endroit où se trouve tous les autres modèles : `{{package_name}}/models_training/`

Il suffit maintenant de modifier le script `2_training.py` pour rajouter notre modèle. En haut du script, rajouter `from {{package_name}}.models_training.utils_tutorial_fr import ModelAuthor` pour importer notre modèle. Et à la ligne 227, mettez en commentaire le modèle `ModelTfidfSvm` et à la place ajoutez le contenu de la cellule suivante:                                              

In [ ]:
model = ModelAuthor(x_col=x_col, 
                    y_col=y_col, 
                    level_save=level_save,
                    multi_label=multi_label,
                    nb_word_sentence=300)

Relancez le script python avec la même commande que précedemment : `python 2_training.py -f texts_train_preprocess_P1.csv --x_col preprocessed_text --y_col author`

Et rechargez le nouveau modèle entrainé avec la cellule suivante (toujours en changeant le nom du modèle chargé):

In [ ]:
model, model_conf = utils_models.load_model('model_author_{YYYY_MM_DD-hh_mm_ss}')

On teste à nouveau notre modèle pour voir sa performance

In [ ]:
df_test_preprocessed['prediction'] = model.predict(df_test_preprocessed['preprocessed_text'])
print('accuracy :', len(df_test_preprocessed[df_test_preprocessed['prediction']==df_test_preprocessed['author']])/len(df_test_preprocessed))

C'est beaucoup mieux!!

Ce petit exemple illustre la manière d'utiliser les frameworks. On part d'un projet contenant tous les outils pour développer rapidement un modèle. On teste quelques modèles inclus pour voir si un correspond à notre cas d'usage. Et, si besoin, on développe son propre modèle en héritant des classes définies dans le projet pour profiter de toutes les fonctions incluses dedans.

Nous allons finalement utiliser un autre outil incorporé dans les frameworks : le démonstrateur. Il est souvent nécessaire de montrer comment fonctionne un modèle à des personnes qui ne l'ont pas développé et le démonstrateur est fait pour cela. Les frameworks utilisent streamlit pour mettre en place ce démonstrateur.

Il suffit de se placer dans le dossier `{{package_name}}/{{package_name}}-scripts/` et de faire `streamlit run 4_demonstrator.py` pour lancer le démonstrateur (il faut peut-être cliquer sur un lien ou le copier-coller dans un navigateur suivant votre configuration).

Dans le menu de gauche, vous pouvez sélectionner le modèle que vous voulez considérer. Il suffit d'entrer le texte que vous voulez tester et appuyer sur predict pour accéder à la prédiction du modèle.